In [1]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
n_gpu = torch.cuda.device_count()

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

In [3]:
import sys
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
sys.path.append("../../../src/")
import data_utils

# Dataset Utilities

In [4]:
william_dir = dict(
    hotel = "../../../data/absa/id/william"
)

william = dict(
    hotel = dict(
        train = data_utils.read_data(path=william_dir["hotel"] + "/train.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=william_dir["hotel"] + "/dev.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=william_dir["hotel"] + "/test.txt",
                                     target_format="aos")
    )
)

# Data Preprocessing 1

1. AOS (ASTE)
    * AO
    * AS
    * A
    * O

2. ACS (TASD)
    * AS
    * CS
    * A
    * C

3. ACOS
    * AO
    * AS
    * CS
    * A
    * O
    * C

In [5]:
# task_tree = {
#     "aos" : ["aos","ao","as",'a','o'],
#     "asc" : ["asc","as","sc",'a','c'],
#     "oasc" : ["oasc","ao","as","sc",'a','o','c']
# }

# all_task = []
# for k,v1 in task_tree.items():
#     if k not in all_task:
#         all_task.append(k)
#     for v2 in v1:
#         if v2 not in all_task:
#             all_task.append(v2)

# print(all_task)

tasks = {
    "single" : ['a', 'o'],
    "simple" : ["ao", "as"],
    "complex" : ["aos"]
}

In [6]:
combination_tasks = [
    tasks["simple"],
    tasks["complex"],
    tasks["single"] + tasks["simple"],
    tasks["single"] + tasks["complex"],
    tasks["simple"] + tasks["complex"],
    tasks["single"] + tasks["simple"] + tasks["complex"]
]

In [7]:
all_task = combination_tasks[-1]
print(all_task)

['a', 'o', 'ao', 'as', 'aos']


In [8]:
from copy import deepcopy

# William (AOS ID)
william_intermediate = dict()

for domain, v1 in william.items():
    william_intermediate[domain] = dict()
    for task in all_task:
        william_intermediate[domain][task] = dict()
        for split in v1.keys():
            ds = william[domain][split]
            ds_copy = deepcopy(ds)
            for i in range(len(ds_copy)):
                # Reduce
                ds_copy[i]["target"] = data_utils.reduce_targets(ds_copy[i]["target"],task)
                # Remove Duplicates
                ds_copy[i]["target"] = data_utils.remove_duplicate_targets(ds_copy[i]["target"])
            william_intermediate[domain][task][split] = ds_copy

# Answer Engineering

In [9]:
mask = "<extra_id_X>"

In [10]:
added_tokens = {
    ',' : "<comma>",
    '(' : "<open_bracket>",
    ')' : "<close_bracket>",
    ';' : "<semicolon>"
}

In [11]:
def construct_answer(targets,se_order):
    if len(targets) == 0:
        return "NULL"
    result = []
    counter = 0
    for t in targets:
        constructed_t = ""
        for se in se_order:
            counter = counter % 100
            constructed_t += ' ' + mask.replace('X',str(counter)) + ' ' + t[data_utils.SENTIMENT_ELEMENT[se]]
            counter += 1
        constructed_t = constructed_t.strip()
        result.append(constructed_t)
    result = " ; ".join(result)
    return result
# def construct_answer(targets,se_order):
#     if len(targets) == 0:
#         return "NULL"
#     result = []
#     for t in targets:
#         constructed_t = []
#         for se in se_order:
#             element = t[data_utils.SENTIMENT_ELEMENT[se]]
#             for k, v in added_tokens.items():
#                 element = element.replace(k,v)
#             constructed_t.append(element)
#         constructed_t = " , ".join(constructed_t)
#         constructed_t = f"( {constructed_t} )"
#         result.append(constructed_t)
#     result = " ; ".join(result)
#     return result

# Prompt Engineering

In [12]:
task_token = {
    "aos" : "<aos>",
    "as" : "<as>",
    "ao" : "<ao>",
    "a" : "<a>",
    "o" : "<o>"
}

In [13]:
def construct_prompt(text,se_order):
    pattern = []
    # for counter, se in enumerate(se_order):
    #     pattern.append(data_utils.SENTIMENT_ELEMENT[se] + " : " + mask.replace('X',str(counter)))
    # pattern = " ,".join(pattern)
    prompt = f"{task_token[se_order]} : "
    # result = text + "| " + pattern
    result = prompt + text
    return result
# def construct_prompt(text,se_order):
#     prompt = []
#     for se in se_order:
#         prompt.append(data_utils.SENTIMENT_ELEMENT[se])
#     prompt = " , ".join(prompt)
#     prompt = f"( {prompt} )"
#     masked_text = text
#     for k, v in added_tokens.items():
#         masked_text = masked_text.replace(k,v)
#     result = masked_text + " | " + prompt
#     return result

# Answer Catch

In [14]:
import re

def catch_answer(output,se_order):
    if output == "NULL":
        return []
    output = output.replace("<pad>",'')
    output = output.replace("</s>",'')
    pattern = r""
    for se in se_order:
        if se != 's':
            pattern += f"<extra_id_\d+>\s*(?P<{data_utils.SENTIMENT_ELEMENT[se]}>[^;]+)\s*"
        else:
            pattern += f"<extra_id_\d+>\s*(?P<{data_utils.SENTIMENT_ELEMENT['s']}>positive|negative|neutral)\s*"
    found = [found_iter.groupdict() for found_iter in re.finditer(pattern,output)]
    for i in range(len(found)):
        for k, v in found[i].items():
            found[i][k] = found[i][k].strip()
    return found
# def catch_answer(output,se_order):
#     if output == "NULL":
#         return []
#     output = output.replace("<pad>",'')
#     output = output.replace("</s>",'')
#     pattern = []
#     for se in se_order:
#         if se != 's':
#             pattern.append(f"\s*(?P<{data_utils.SENTIMENT_ELEMENT[se]}>[^;]+)\s*")
#         else:
#             pattern.append(f"\s*(?P<{data_utils.SENTIMENT_ELEMENT['s']}>positive|negative|neutral)\s*")
#     pattern = ','.join(pattern)
#     pattern = f"\({pattern}\)"
#     found = [found_iter.groupdict() for found_iter in re.finditer(pattern,output)]
#     for i in range(len(found)):
#         for k, v in found[i].items():
#             found[i][k] = found[i][k].strip()
#     return found

# Data Preprocessing 2

# Prepare Tokenized Dataset

In [15]:
encoding_args = {
    "max_length" : 128,
    "padding" : True,
    "truncation" : True,
    "return_tensors" : "pt"
}

In [16]:
tokenizer_id = AutoTokenizer.from_pretrained("google/mt5-base")

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [17]:
tokenizer_id.add_tokens(list(task_token.values()))

5

In [18]:
def encode_id(dataset):
    result = tokenizer_id(dataset["input"], text_target=dataset["output"], **encoding_args)
    return result

In [19]:
from datasets import Dataset

def create_data_2(tasks):
    william_2 = dict()
    for domain, v1 in william_intermediate.items():
        william_2[domain] = {
            "train" : [], # basic task
            "val" : [], # complex task
            "test" : [] # complex task
        }
        # TRAIN
        for basic_task in tasks:
            for el in william_intermediate[domain][basic_task]["train"]:
                william_2[domain]["train"].append({
                        "input" : construct_prompt(el["text"],basic_task),
                        "output" : construct_answer(el["target"],basic_task),
                        "task" : basic_task
                    })
        # VAL
        for el in william_intermediate[domain]["aos"]["val"]:
            william_2[domain]["val"].append({
                    "input" : construct_prompt(el["text"],"aos"),
                    "output" : construct_answer(el["target"],"aos"),
                    "task" : "aos"
                })
        # TEST
        for el in william_intermediate[domain]["aos"]["test"]:
            william_2[domain]["test"].append({
                    "input" : construct_prompt(el["text"],"aos"),
                    "output" : construct_answer(el["target"],"aos"),
                    "task" : "aos"
                })
        william_2[domain]["train"] = Dataset.from_list(william_2[domain]["train"])
        william_2[domain]["val"] = Dataset.from_list(william_2[domain]["val"])
        william_2[domain]["test"] = Dataset.from_list(william_2[domain]["test"])
    
    william_tok = dict()
    for domain, v1 in william_2.items():
        william_tok[domain] = dict()
        for split, v2 in v1.items():
            if split != "test":
                william_tok[domain][split] = william_2[domain][split].map(encode_id,batched=True,remove_columns=["input","output","task"])
                william_tok[domain][split].set_format("torch")
            else:
                william_tok[domain][split] = encode_id(william_2[domain][split])
    
    return william_2, william_tok

# Data Collator

## Indo

In [20]:
from transformers import DataCollatorForSeq2Seq

data_collator_id = DataCollatorForSeq2Seq(tokenizer=tokenizer_id)

# Compute Metrics

In [21]:
from transformers import EvalPrediction
from evaluation import recall, precision, f1_score, summary_score
from typing import List, Dict, Tuple
import numpy as np

def seperate_target_prediction_per_task(predictions:List[List[Dict]],targets:List[List[Dict]],tasks:List) -> Tuple[Dict[str,List],Dict[str,List]]:
    per_task_targets = {}
    per_task_predictions = {}
    for target, prediction, task in zip(targets,predictions,tasks):
        if task not in per_task_targets.keys():
            per_task_targets[task] = []
        if task not in per_task_predictions.keys():
            per_task_predictions[task] = []
        per_task_targets[task].append(target)
        per_task_predictions[task].append(prediction)
    return per_task_targets, per_task_predictions

def preprocess_eval_preds(eval_preds:EvalPrediction,decoding_args:Dict[str,str],tokenizer:AutoTokenizer):
    input_ids = eval_preds.inputs
    target_ids = eval_preds.label_ids
    pred_ids = eval_preds.predictions

    # In case the model returns more than the prediction logits
    if isinstance(input_ids, tuple):
        input_ids = input_ids[0]
    if isinstance(target_ids, tuple):
        target_ids = target_ids[0]
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]
    
    input_ids = np.argmax(input_ids,axis=-1) if len(input_ids.shape) == 3 else input_ids # in case not predict with generate
    target_ids = np.argmax(target_ids,axis=-1) if len(target_ids.shape) == 3 else target_ids # in case not predict with generate
    prediction_ids = np.argmax(pred_ids,axis=-1) if len(pred_ids.shape) == 3 else pred_ids # in case not predict with generate

    input_ids = [[token for token in row if token != -100] for row in input_ids]
    target_ids = [[token for token in row if token != -100] for row in target_ids]
    prediction_ids = [[token for token in row if token != -100] for row in prediction_ids]

    inputs = tokenizer.batch_decode(input_ids,**decoding_args)
    targets = tokenizer.batch_decode(target_ids,**decoding_args)
    predictions = tokenizer.batch_decode(prediction_ids,**decoding_args)

    return inputs, targets, predictions

def compute_metrics(eval_preds:EvalPrediction,decoding_args:Dict[str,str],tokenizer:AutoTokenizer,tasks:List) -> Dict[str,float]: # MAY NOT BE SUFFICIATE FOR CAUSAL LM
        """
        ### DESC
            Method to compute the metrics.
        ### PARAMS
        * eval_preds: EvalPrediction instance from training.
        * decoding_args: Decoding arguments.
        ### RETURN
        * metrics: Dictionary of metrics.
        """
        inputs, targets, predictions = preprocess_eval_preds(eval_preds,decoding_args,tokenizer)

        print("INPUTS >>",inputs[0])
        print("TARGETS >>",targets[0])
        print("PREDS >>",predictions[0])

        targets = [catch_answer(text,task) for text,task in zip(targets,tasks) if task != "non_absa"]
        predictions = [catch_answer(text,task) for text,task in zip(predictions,tasks) if task != "non_absa"]

        per_task_targets, per_task_predictions = seperate_target_prediction_per_task(predictions, targets, tasks)
        
        metrics = {}

        metrics["overall_recall"] = recall(predictions,targets)
        metrics["overall_precision"] = precision(predictions,targets)
        metrics["overall_f1_score"] = f1_score(predictions,targets)

        for task in per_task_targets.keys():
            if task == "non_absa":
                continue
            metrics[f"{task}_recall"] = recall(per_task_predictions[task],per_task_targets[task])
            metrics[f"{task}_precision"] = precision(per_task_predictions[task],per_task_targets[task])
            metrics[f"{task}_f1_score"] = f1_score(per_task_predictions[task],per_task_targets[task])
        
        return metrics

# Train Arguments

In [22]:
from transformers import Seq2SeqTrainingArguments

train_args = {
    "num_train_epochs": 10,
    "learning_rate": 3e-4,
    "save_total_limit": 2,
    "gradient_accumulation_steps": 2,
    "per_device_train_batch_size": 8//n_gpu,
    "per_device_eval_batch_size": 8//n_gpu,
    "save_strategy": "epoch",
    "evaluation_strategy": "epoch",
    "logging_strategy" : "epoch",
    "metric_for_best_model": "overall_f1_score",
    "load_best_model_at_end": True,
    "adam_epsilon": 1e-08,
    "output_dir": "./output",
    "logging_dir" : "./output/log",
    "include_inputs_for_metrics" : True
}

train_args = Seq2SeqTrainingArguments(**train_args)

# Train

In [23]:
import torch
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


In [24]:
from transformers import Seq2SeqTrainer

# trainer = {
#     "peng" : {},
#     "wan" : {},
#     "zhang" : {},
#     "william" : {}
# }

decoding_args = {
    "skip_special_tokens" : False
}

def preprocess_logits_for_metrics(logits, targets):
    pred_logits = logits[0] if isinstance(logits,tuple) else logits
    pred_ids = torch.argmax(pred_logits, dim=-1)
    return pred_ids, targets

In [25]:
from tqdm import tqdm

def generate_predictions(model,tokenizer,tokenized:torch.Tensor,device:torch.device=torch.device("cpu"),batch_size:int=16,max_len:int=128,decoding_args:Dict={}) -> List[str]:
    # Data loader
    input_ids_data_loader = torch.utils.data.DataLoader(tokenized["input_ids"],
                        batch_size=batch_size,shuffle=False)
    attention_mask_data_loader = torch.utils.data.DataLoader(tokenized["attention_mask"],
                        batch_size=batch_size,shuffle=False)
    # Predict
    model = model
    tokenizer = tokenizer
    tensor_predictions = []
    with torch.no_grad():
        for input_ids, attention_mask in tqdm(zip(input_ids_data_loader,attention_mask_data_loader)):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            tensor_predictions.extend(model.generate(input_ids=input_ids,attention_mask=attention_mask,max_length=max_len,pad_token_id=tokenizer.pad_token_id,eos_token_id=tokenizer.eos_token_id).cpu())
            input_ids = input_ids.cpu()
            attention_mask = attention_mask.cpu()
    tensor_predictions = [[token for token in row if token != -100] for row in tensor_predictions]
    predictions = tokenizer.batch_decode(tensor_predictions,**decoding_args)
    return predictions

In [26]:
import json

def save_result(str_preds_,preds,targets,filename):
    result = []
    str_preds = [el.replace("<pad>",'').replace("</s>",'') for el in str_preds_]
    assert len(str_preds) == len(preds) == len(targets)
    for i in range(len(str_preds)):
        result.append({
            "str_pred" : str_preds[i],
            "pred" : preds[i],
            "target" : targets[i]
        })
    
    with open(filename,'w') as fp:
        json.dump(result,fp)
    return result

# William Hotel

In [27]:
william_2, william_tok = create_data_2(all_task)

In [29]:
william_2["hotel"]["train"]["input"][-2]

'<aos> : sangat kecewa dengan kamar dan pelayanan stafnya !'

In [33]:
# for combo_task in combination_tasks:
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")
model.resize_token_embeddings(len(tokenizer_id))
model.to(device)
trainer = Seq2SeqTrainer(
        model = model,
        args = train_args,
        tokenizer = tokenizer_id,
        data_collator = data_collator_id,
        train_dataset = william_tok["hotel"]["train"],
        eval_dataset = william_tok["hotel"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_id,william_2["hotel"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer.train()

# str_preds = generate_predictions(model, tokenizer_id, william_2["hotel"]["test"]["input"], device, decoding_args)
# preds = [catch_answer(el,"aos") for el in str_preds]
str_preds = generate_predictions(model, tokenizer_id, william_tok["hotel"]["test"], device, 16, 128, decoding_args)
preds = [catch_answer(el,"aos") for el in str_preds]
targets = [catch_answer(el,"aos") for el in william_2["hotel"]["test"]["output"]]
score = summary_score(preds,targets)
print(f"Score for {all_task} >>", score)
fname = "one_token"
result = save_result(str_preds, preds, targets, fname + "_pred.json")
with open(fname + "_score.json", 'w') as fp:
    json.dump(score,fp)

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 15000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 9370
  Number of trainable parameters = 582390528
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 31.75 GiB total capacity; 10.90 GiB already allocated; 9.94 MiB free; 11.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
!rm -rf ./output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
